In [1]:
import pickle
import argparse
import os
import numpy as np
import pandas as pd

import torch
from torch.utils.data import ConcatDataset
from torch.utils.data import DataLoader
from torch.nn import CrossEntropyLoss
from torch.optim import Adam
from sklearn.metrics import balanced_accuracy_score

from utils.training import train
from utils.evaluation import score
from utils.data import Data, SpikeDetectionDataset


/home/theo/anaconda3/envs/neuropoly/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
expe_seed = 0
method = 'Fukumori'
parameters = [0]
sfreq = 100  # Sampling frequency
batch_size = 16


In [3]:
path_root = 'database/'
dataset = Data(path_root, 'spikeandwave', 'EEG', True)
all_dataset = dataset.all_datasets()

2022-05-11 15:40:48.095 | INFO     | utils.data:get_all_datasets:193 - Recover data for Neuropoly_MEEG_database
2022-05-11 15:41:49.298 | INFO     | utils.data:get_dataset:167 - Label creation: No Spike / Spikes mapped on labels [0]


In [4]:
n_epochs = 100
patience = 10

assert method in ("Fukumori")

num_workers = 0  # Number of processes to use for the data loading process; 0 is the main Python process

results = []


In [5]:
data = all_dataset[0]['Neuropoly_MEEG_database']


In [6]:
labels = all_dataset[1]['Neuropoly_MEEG_database']

In [7]:
labels = np.array([labels for _ in range(50)]).T

In [8]:
data = data.reshape(data.shape[0]*data.shape[1], data.shape[2])
data = data[:, :, np.newaxis]
labels = labels.reshape(labels.shape[0]*labels.shape[1])

In [9]:
from sklearn.model_selection import train_test_split

data_train, data_test, labels_train, labels_test = train_test_split(data, labels )
data_train, data_val, labels_train, labels_val = train_test_split(data_train, labels_train)

In [10]:
data.shape

(32750, 201, 1)

In [11]:
dataset_train = SpikeDetectionDataset(data_train, labels_train)
dataset_val = SpikeDetectionDataset(data_val, labels_val)
dataset_test = SpikeDetectionDataset(data_test, labels_test)

In [12]:
data.shape

(32750, 201, 1)

In [13]:
from torch.utils.data import DataLoader

loaders_train = DataLoader(dataset_train, batch_size=batch_size)
loaders_val = DataLoader(dataset_val, batch_size=batch_size)
loaders_test = DataLoader(dataset_test, batch_size=batch_size)


In [14]:
input_size = loaders_train.dataset[0][0].shape[1]



In [28]:
import utils.model as model
import utils.training as training
import importlib
importlib.reload(model)
importlib.reload(training)

if method == "Fukumori":
    model = model.fukumori2021RNN(input_size=input_size)

lr = 1e-3  # Learning rate
optimizer = Adam(params=model.parameters(), lr=lr)

criterion = CrossEntropyLoss()

In [29]:
# Train Model
best_model, history = training.train(
    model,
    method,
    loaders_train,
    loaders_val,
    optimizer,
    criterion,
    parameters,
    n_epochs,
    patience,
)


epoch 	 train_loss 	 valid_loss 	 train_perf 	 valid_perf
--------------------------------------------------------------------------------
1 	 0.0000 	 0.0000 	0.0000 	 0.0000
best val loss inf -> 0.0000
2 	 0.0000 	 0.0000 	0.0000 	 0.0000
3 	 0.0000 	 0.0000 	0.0000 	 0.0000
4 	 0.0000 	 0.0000 	0.0000 	 0.0000
5 	 0.0000 	 0.0000 	0.0000 	 0.0000
6 	 0.0000 	 0.0000 	0.0000 	 0.0000
7 	 0.0000 	 0.0000 	0.0000 	 0.0000
8 	 0.0000 	 0.0000 	0.0000 	 0.0000
9 	 0.0000 	 0.0000 	0.0000 	 0.0000
10 	 0.0000 	 0.0000 	0.0000 	 0.0000
11 	 0.0000 	 0.0000 	0.0000 	 0.0000
Stop training at epoch 11
Best val loss : 0.0000


In [21]:
A =np.array([0 ,1])
np.where(A==1)[0]

array([1])